In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 

In [16]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, padding='post', truncating='post', maxlen=40)

X_val_sequences = tokenizer.texts_to_sequences(X_val)
X_val_padded = pad_sequences(X_val_sequences, padding='post', truncating='post', maxlen=40)

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, padding='post', truncating='post', maxlen=40)

In [17]:
len(word_index)

23870

In [18]:
from tensorflow import keras
from tensorflow.keras import layers

In [56]:
embedding_dim = 128
vocab_size = len(word_index) + 1

model = keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim),
    layers.Bidirectional(layers.LSTM(128, dropout=0.2, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(64, dropout=0.15)),
    layers.BatchNormalization(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(rate=0.1),
    layers.Dense(1, activation='sigmoid')
])

In [44]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 128)         3055488   
                                                                 
 bidirectional_10 (Bidirecti  (None, None, 128)        98816     
 onal)                                                           
                                                                 
 bidirectional_11 (Bidirecti  (None, 64)               41216     
 onal)                                                           
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                      

In [25]:
from tensorflow.keras import callbacks, metrics

In [49]:
METRICS = [
    metrics.BinaryAccuracy(name='accuracy'),
    metrics.Precision(name='precision'),
    metrics.Recall(name='recall')
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4), 
    loss='binary_crossentropy', 
    metrics=METRICS
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    min_delta=1e-6,
    patience=10,
    verbose=1,
    restore_best_weights=True
)

reduce_learning_rate = callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=10, 
    min_lr=1e-6, 
    verbose=1
)

In [27]:
import tensorflow as tf

In [50]:
history = model.fit(
    tf.constant(X_train_padded), 
    tf.constant(y_train), 
    validation_data=(tf.constant(X_val_padded), tf.constant(y_val)),
    batch_size=32,
    epochs=30,
    callbacks=[early_stopping, reduce_learning_rate],
)

Epoch 1/30
656/656 [==============================] - 25s 30ms/step - loss: 0.4709 - accuracy: 0.7638 - precision: 0.7554 - recall: 0.7801 - val_loss: 0.4181 - val_accuracy: 0.8042 - val_precision: 0.9356 - val_recall: 0.6533 - lr: 1.0000e-04
Epoch 2/30
656/656 [==============================] - 21s 31ms/step - loss: 0.2369 - accuracy: 0.9057 - precision: 0.9007 - recall: 0.9119 - val_loss: 0.3576 - val_accuracy: 0.8516 - val_precision: 0.7844 - val_recall: 0.9697 - lr: 1.0000e-04
Epoch 3/30
656/656 [==============================] - 25s 38ms/step - loss: 0.1425 - accuracy: 0.9479 - precision: 0.9485 - recall: 0.9474 - val_loss: 0.5452 - val_accuracy: 0.8189 - val_precision: 0.7413 - val_recall: 0.9795 - lr: 1.0000e-04
Epoch 4/30
656/656 [==============================] - 25s 38ms/step - loss: 0.0949 - accuracy: 0.9668 - precision: 0.9666 - recall: 0.9669 - val_loss: 0.3155 - val_accuracy: 0.8963 - val_precision: 0.8932 - val_recall: 0.9003 - lr: 1.0000e-04
Epoch 5/30
656/656 [========